## Langchain Basic Usage (LLM + ChatModel only)
Langchain的框架下面有主要分為兩種Model
- LLM
- Chat

由於Langchain並不提供任何Model，所以LLM相關的library都不在langchain-core裡面

#### Langchain最基本的用法
`Prompt -> LLM -> Response`

##### Loading environment variables

In [3]:
%load_ext dotenv
%dotenv

import os

##### Basic Prompts

In [ ]:
# Everything starts with LLM
from langchain_core.messages import HumanMessage
msg = HumanMessage(content='茶餐廳是什麼？')

##### 使用ChatModel (e.g. TogetherAI)

In [ ]:
from langchain_together import ChatTogether
model = ChatTogether(
    together_api_key=os.environ['KEY_TOGETHERAI'],
    model="Qwen/Qwen1.5-72B-Chat",
)

In [ ]:
res = model.invoke([msg])

##### 利用output_parser將output變得更易讀

In [ ]:
print(res)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

p = StrOutputParser()
# print(type(p))
print(p.invoke(res))

##### 加入SystemMessage (Instruction)

In [ ]:
from langchain_core.messages import SystemMessage
system_message = """
必須以繁體中文回答
回答時只以','分隔每個項目，並以最簡短的方式回答，例如：
一,二,三,...
"""
human_message = """
請列舉茶餐廳不多過10種最熱門的食物，不肯定的不要回答
"""
msg_list = [
    SystemMessage(content=system_message),
    HumanMessage(content=human_message)
]

In [ ]:
res = model.invoke(msg_list)

In [ ]:
print(p.invoke(res))

##### Structured Data Parser
https://python.langchain.com/v0.2/docs/concepts/#output-parsers

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser

csvp = CommaSeparatedListOutputParser()
parsed_res = csvp.invoke(res)
print(type(parsed_res))
print(parsed_res)

##### 用PromptTemplate更方便

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("human", "請列舉{place}不多過10種最熱門的食物，不肯定的不要回答"),
])

prompt_value = prompt_template.invoke(
    {
        'place': '日本'
    }
)

In [ ]:
print(prompt_value)

In [ ]:
japan_csv = csvp.invoke(model.invoke(prompt_value))

In [ ]:
print(japan_csv)

##### 以steaming的方式回答

In [ ]:
author_system_message = """你是一個日本輕小說作家，寫作風格要模仿{author}，請以用戶的名字命名主角，並以提供的關鍵字去寫一個大約400字的短篇故事。故事必須以繁體中文編寫。"""
prompt_template2 = ChatPromptTemplate.from_messages([
    ("system", author_system_message),
    ("ai", "請問你怎樣稱呼？"),
    ("human", "我的名字叫{username}"),
    ("ai", "你好{username}，請問你想以什麼作題材寫這個故事呢？"),
    ("human", "我想以{topics}為題材寫這個故事"),
])
prompt_value = prompt_template2.invoke(
    {
        "author": "川原礫",
        "username": "彌豆子",
        "topics": "鬼，捉鬼，親情，拯救世界"
    }
)
print(prompt_value)

In [ ]:
for res in model.stream(prompt_value):
    print(res.content, end="|")

#### LCEL: LangChain Expression Language
怎樣用LCEL簡化日本輕小說家的Chain呢？ 

In [ ]:
jnovel = prompt_template2 | model | p
res = jnovel.invoke(
    {
        "author": "支倉凍砂",
        "username": "亨利",
        "topics": "狼，香辛料，北方，旅行商人"
    })

In [ ]:
print(p.parse(res))

In [ ]:
jnovel2 = prompt_template2 | model | p
for res in jnovel2.stream(
    {
        "author": "支倉凍砂",
        "username": "亨利",
        "topics": "狼，香辛料，北方，旅行商人"
    }):
    print(res, end="")

##### 問題：Print的為何不再是res.content?

https://python.langchain.com/v0.2/docs/how_to/streaming/

### 偉大的LangChain: Runnable
https://python.langchain.com/v0.2/docs/concepts/#runnable-interface

#### Bonus Track: Tools Calling 😎

In [6]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

tools = [add, multiply]

In [10]:
from langchain_core.pydantic_v1 import BaseModel, Field

# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class Add(BaseModel):
    """Add two integers together."""
    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

class Multiply(BaseModel):
    """Multiply two integers together."""
    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

In [7]:
# mistralai/Mixtral-8x22B-Instruct-v0.1
from langchain_openai import ChatOpenAI

better_model = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["KEY_TOGETHERAI"],
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",)

model_with_tools = better_model.bind_tools(tools)

In [12]:
query = "What is 3 * 12? Also, what is 11 + 49?"

model_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_c9h4fp3hkehqluo2e4c7hh89'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_jbh3b9zk93vca027p231cjgk'}]

In [14]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage(query)]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_h91xqdgcbuvujsfw14qr1tam', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_crxrdmkmz3azw0jmj7x79fkt', 'function': {'arguments': '{"a":11,"b":49}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 304, 'total_tokens': 388}, 'model_name': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-59feef45-3db1-4be2-846d-48066dad7e5f-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_h91xqdgcbuvujsfw14qr1tam'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_crxrdmkmz3azw0jmj7x79fkt'}], usage_metadata={'input_tokens': 304, 'output_tokens': 84, 'total_tokens': 388}),
 ToolMessage(content='36', tool_call_id='call_h91xqdgcbuvujsfw14qr1tam'),
 To

In [15]:
model_with_tools.invoke(messages)

AIMessage(content='', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 438, 'total_tokens': 439}, 'model_name': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'system_fingerprint': None, 'finish_reason': 'eos', 'logprobs': None}, id='run-f9b725bb-bcde-450b-a888-9a32edff03f1-0', usage_metadata={'input_tokens': 438, 'output_tokens': 1, 'total_tokens': 439})